In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Downloading the info

#### USOS_ZONAS_6_7

In [2]:
USOS_ZONA_7 = gpd.read_file("USOS_ZONA_7.shp")
USOS_ZONA_6 = gpd.read_file("USOS_ZONA_6.shp")

In [5]:
USOS_ZONA_7.columns

Index(['gid', 'id', 'distrito', 'shape_leng', 'shape_area', 'UNuso',
       'geometry'],
      dtype='object')

#### DENUE

In [4]:
schema = "denue"
table = "denue_2022"

manzana = "manzana"
municipio = "municipio"
id = "id"
latitud = "latitud"
longitud = "longitud"
tipoCenCom = "tipoCenCom"
edificio = "edificio"
edificio_e = "edificio_e"
nom_CenCom = "nom_CenCom"

query_censo = f"""
SELECT 
  "{manzana}",
  "{municipio}",
  "{id}",
  "{latitud}",
  "{longitud}",
  "{tipoCenCom}",
  "{edificio}",
  "{edificio_e}",
  "{nom_CenCom}",
  "geometry"
FROM {schema}.{table}"""

Denue = aup.gdf_from_query(query_censo, geometry_col='geometry')
print(Denue.shape)
Denue.head(2)

(5528698, 10)


,manzana,municipio,id,latitud,longitud,tipoCenCom,edificio,edificio_e,nom_CenCom,geometry
0,056,Aguascalientes,18016,21.901488,-102.289962,None,None,None,None,POINT (-102.28996 21.90149)
1,030,Jesús María,49318,21.992029,-102.288802,None,None,None,None,POINT (-102.28880 21.99203)


In [4]:
Denue.columns

Index(['manzana', 'municipio', 'id', 'latitud', 'longitud', 'tipoCenCom',
       'edificio', 'edificio_e', 'nom_CenCom', 'geometry'],
      dtype='object')

In [5]:
Denue.isna().sum()

manzana             0
municipio           0
id                  0
latitud             0
longitud            0
tipoCenCom    5115339
edificio      5094025
edificio_e    5082311
nom_CenCom    5115452
geometry            0
dtype: int64

In [4]:
Denue["tipoCenCom"].unique()

array([None, 'CENTRO Y PLAZA COMERCIAL', 'OTRO CONGLOMERADO',
       'HOSPITAL, CENTRO MEDICO O CLIN', 'EDIFICIO COMERCIAL Y DE SERVIC',
       'CENTRAL DE ABASTO', 'MERCADO PUBLICO', 'PARQUE INDUSTRIAL',
       'CIUDAD INDUSTRIAL', 'AEROPUERTO', 'TIANGUIS, BAZAR O PULGA',
       'PASAJE Y ANDADOR COMERCIAL', 'CORREDOR INDUSTRIAL',
       'PARQUE RECREATIVO', 'CENTRAL CAMIONERA', 'ZONA INDUSTRIAL',
       'EDIFICIO HABITACIONAL Y MIXTO', 'CORREDORES TURÍSTICOS',
       'ESCUELAS', 'HOTELES', 'OTROS CONJUNTOS COMERCIALES'], dtype=object)

In [11]:
Denue["nom_CenCom"].unique()

array([None, 'TORRE PLAZA BOSQUES', 'DEL VALLE DE AGUASCALIENTES', ...,
       'SERVICIO LUFER', 'CIUDAD ADMINISTRATIVO', 'RECAUDACION DE RENTAS'],
      dtype=object)

#### BUILDINGS

In [ ]:
import gzip ## No temgo RAM Suficiente.

with gzip.open("843_buildings.csv.gz", 'rt') as f:
    df = pd.read_csv(f)

df.head(3)

### Pruebas de conteo de actividades 

In [5]:
# Función para obtener el tipoCenCom más común, manejando vacíos
def tipoCenCom_predominante(x):
    if x.dropna().empty:
        return None  # O puedes poner 'Sin datos', 'Desconocido', etc.
    else:
        return x.value_counts().idxmax()

# Ahora agrupamos usando la función segura
grupo_manzana = Denue.groupby('manzana')['tipoCenCom'] \
    .agg(tipoCenCom_predominante) \
    .reset_index()

grupo_manzana.head()

,manzana,tipoCenCom
0,001,MERCADO PUBLICO
1,002,MERCADO PUBLICO
2,003,MERCADO PUBLICO
3,004,MERCADO PUBLICO
4,005,MERCADO PUBLICO


In [6]:
# Cuántas manzanas tienen al menos un tipoCenCom no nulo
manzanas_con_datos = Denue.dropna(subset=['tipoCenCom'])['manzana'].nunique()

# Cuántas manzanas en total
total_manzanas = Denue['manzana'].nunique()

print(f"Manzanas con datos de tipoCenCom: {manzanas_con_datos} de {total_manzanas}")

Manzanas con datos de tipoCenCom: 136 de 314


In [7]:
# Ver qué sistema de refrencia tiene cada uno:
print(USOS_ZONA_7.crs) 
print(Denue.crs)

EPSG:32613
epsg:4326


In [ ]:
Denue = Denue.to_crs(USOS_ZONA_7.crs) # Reproyectar

In [ ]:
# Join espacial de Denue (puntos) dentro de USOS_ZONA_7 (polígonos)
interseccion_7 = gpd.sjoin(Denue, USOS_ZONA_7, how='left', predicate='within')

interseccion_7.head(3)